In [7]:
##################################################
#스크래핑할 웹 사이트 URL 
#url <- 'https://www.imdb.com/search/title/?count=100&release_date=2019,2019&title_type=feature'
# https://www.imdb.com/search/title/?count=100&release_date=2018,2018&title_type=feature
#
#http://www.imdb.com/search/title?count=100&release_date=2017,2017&title_type=feature
# 스크래핑할 데이터 - rank, title, description, runtime, genre, rating, metascore, votes, gross_earning_in_Mil, director, actor


# 스크래핑한 데이터 pandas의 data.frame으로 변환하여 2017bestfilms.csv파일로 저장  


from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

try_to_text = lambda x : x.text if x else None
url = 'https://www.imdb.com/search/title/?count=100&release_date=2017,2017&title_type=feature'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
df = pd.DataFrame()

for ele in soup.select('.lister-item-content'):
    # 100개의 (각 영화의 정보를 가지고 있는)엘리먼트에 대한 loop
    # 딕셔너리에 엘리먼트로부터 수집한 영화 정보를 할당하고 dataframe에
    # 딕셔너리를 행으로 추가

    # 처리가 간단한 정보는 딕셔너리에 바로 할당
    row = {
        'rank': ele.select_one('.lister-item-index').text.replace('.', ''),
        'title': ele.select_one('.lister-item-header > a').text,
        'description': ele.select('p.text-muted')[1].text.replace('\n', ' '),
        'runtime': ele.select_one('.runtime').text[:-4],
        'genre': ele.select_one('.genre').text,
        'rating': ele.select_one('.ratings-imdb-rating > strong').text,
        'metascore': try_to_text(ele.select_one('.metascore')),
        'votes': None,
        'gross': None,
    }

    # 처리가 복잡한 정보는 파싱하여 딕셔너리에 할당
    votes_n_gross = ele.select_one('.sort-num_votes-visible').get_text()
    director_n_star = ele.select_one('p:nth-of-type(3)').get_text()
    pattern = re.compile(r'[\s\n$M]')
    for data in votes_n_gross.split('|'):
        key, val = re.sub(pattern, '', data).split(':')
        row[key.lower()] = float(val.replace(',', ''))
    for data in director_n_star.split('|'):
        key, val = data.replace('\n', '').split(':')
        if 'Directors' in key:
            key = 'director'
        row[key.lower().strip()] = val.strip()

    # 딕셔너리를 dataframe의 행으로 추가
    df = df.append(row, ignore_index=True)
    

# 1. csv 저장
#df.to_csv('C:\\Users\\student\\mypy\\data\\output\\movie2017_info.csv')


###########################################################
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
 

#request 모듈 사용
url = 'http://www.imdb.com/search/title?count=100&release_date=2017,2017&title_type=feature'
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

 
#각 컬럼 별 리스트로 저장
box = soup.select('.lister-item-content')  

#rank
rank_list = []
for i in box:
    textline = i.select_one('h3 > .lister-item-index.unbold.text-primary')
    textline = textline.text
    textline = re.sub('[\n.]','',textline)
    rank_list.append(int(textline))
print(len(rank_list))
print(rank_list)

 

#title
title_list = []
for i in box:
    textline = i.select_one('h3 > a')
    textline = textline.text
    title_list.append(textline)
print(len(title_list))
print(title_list)

 

#description
description_list = []
for i in box:
    textline = i.select_one('p.text-muted:nth-of-type(2)')
    textline = textline.text
    textline = re.sub('[\n]','',textline)
    description_list.append(textline)
print(len(description_list))
print(description_list)

 

#runtime
runtime_list = []
for i in box:
    textline = i.select_one('p > .runtime')
    textline = textline.text
    textline = re.sub('[min]', '',textline)
    runtime_list.append(int(textline))
print(len(runtime_list))
print(runtime_list)

 

# genre
genre_list = []
for i in box:
    textline = i.select_one('p > .genre')
    #print(textline)
    textline = textline.text
    textline = re.sub('[\n ]', '',textline)
    genre_list.append(textline)
print(len(genre_list))
print(genre_list)
 

#rating
rating_list = []
for i in box:
    textline = i.select_one('.inline-block.ratings-imdb-rating').attrs.get("data-value")
    rating_list.append(float(textline))
print(len(rating_list))
print(rating_list)

 

#metascore
metascore_list = []
for i in box:
    textline = i.select_one('.metascore')
    #print(textline)
    if textline == None:
        metascore_list.append(0)
        continue
    textline = textline.text
    metascore_list.append(int(textline))
print(len(metascore_list))
print(metascore_list)

 

#vote
vote_list = []
for i in box:
    #textline = i.select_one("p.sort-num_votes-visible > span:nth-child(5)").attrs.get("data-value",0)
    try:
        textline = i.select_one('p.sort-num_votes-visible > span:nth-child(2)').attrs.get("data-value")
    except AttributeError:
        vote_list.append(0)
        continue
    textline = re.sub(',','',textline)
    vote_list.append(int(textline))
print(len(vote_list))
print(vote_list)
 

#gross_earning_in_Mil
gross_earning_in_Mil_list = []
for i in box:
    #textline = i.select_one("p.sort-num_votes-visible > span:nth-child(5)").attrs.get("data-value",0)
    try:
        textline = i.select_one('p.sort-num_votes-visible > span:nth-child(5)').attrs.get("data-value")
    except AttributeError:
        gross_earning_in_Mil_list.append(0)
        continue
    textline = re.sub(',','',textline)
    gross_earning_in_Mil_list.append(int(textline))    

print(len(gross_earning_in_Mil_list))
print(gross_earning_in_Mil_list)

 

#actor
actor_list = []
for i in box:
    textline = i.select_one('div.lister-item-content > p:nth-child(5)').get_text()
    textline = re.sub('[\n]','',textline)
    list_se = textline.split('Stars:')
    actor_list.append(list_se[1])    

print(len(actor_list))
print(actor_list)     

 

#director
director_list = []
for i in box:
    textline = i.select_one('div.lister-item-content > p:nth-child(5) > a:nth-child(1)')
    #print(textline)
    textline = textline.text
    textline = re.sub('[\n ]', '',textline)
    director_list.append(textline)    

print(len(director_list))
print(director_list)      

 

 

#영화 데이터 프레임 생성
movie_df = pd.DataFrame ([ rank_list, title_list, description_list, runtime_list, genre_list, rating_list, metascore_list, vote_list, gross_earning_in_Mil_list, director_list, actor_list])
columns = ['rank', 'title', 'description', 'runtime', 'genre', 'rating', 'metascore', 'votes', 'gross_earning_in_Mil', 'director', 'actor']
movie_df = movie_df.T

#컬럼 설정
movie_df.columns = columns

#인덱스 설정
movie_df.set_index(['rank'],inplace=True)
#object -> int 또는 float 형변환
movie_df[['runtime','metascore','votes','gross_earning_in_Mil']] = movie_df[['runtime','metascore','votes','gross_earning_in_Mil']].astype(int)
movie_df['rating'] = movie_df['rating'].astype(float)

100
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
100
['Animal Crackers', 'Three Billboards Outside Ebbing, Missouri', 'Blade Runner 2049', 'The Post', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'It', 'Justice League', 'Guardians of the Galaxy Vol. 2', 'Thor: Ragnarok', 'The Greatest Showman', 'Call Me by Your Name', 'Dunkirk', 'Omerta', 'Baby Driver', 'Get Out', "Devil's Gate", 'El guardián invisible', 'Spider-Man: Homecoming', "The Hitman's Bodyguard", 'Star Wars: Episode VIII - The Last Jedi', 'Beauty and the Beast', 'Wonder Woman', 'Jumanji: Welcome to the Jungle', 'Logan', 'Lady Bird', 'Baywatch', 'Alien: Covenant', 'Kong: Skull Is

AttributeError: 'NoneType' object has no attribute 'text'